In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
calendar = pd.read_csv('../data/calendar.csv')
sales_train_val = pd.read_csv('../data/sales_train_validation.csv')
sales_train_eval = pd.read_csv('../data/sales_train_evaluation.csv')
sell_prices = pd.read_csv('../data/sell_prices.csv')

### 01 Preprocessing Data
#### 1.1 Calendar Data

In [3]:
display(calendar.info())
display(calendar.head().T)
display(calendar.describe().T)
display(calendar.isnull().sum().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


None

,0,1,2,3,4
date,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02
wm_yr_wk,11101,11101,11101,11101,11101
weekday,Saturday,Sunday,Monday,Tuesday,Wednesday
wday,1,2,3,4,5
month,1,1,1,2,2
year,2011,2011,2011,2011,2011
d,d_1,d_2,d_3,d_4,d_5
event_name_1,NaN,NaN,NaN,NaN,NaN
event_type_1,NaN,NaN,NaN,NaN,NaN
event_name_2,NaN,NaN,NaN,NaN,NaN


,count,mean,std,min,25%,50%,75%,max
wm_yr_wk,1969.0,11347.086338,155.277043,11101.0,11219.0,11337.0,11502.0,11621.0
wday,1969.0,3.997461,2.001141,1.0,2.0,4.0,6.0,7.0
month,1969.0,6.325546,3.416864,1.0,3.0,6.0,9.0,12.0
year,1969.0,2013.288471,1.580198,2011.0,2012.0,2013.0,2015.0,2016.0
snap_CA,1969.0,0.330117,0.470374,0.0,0.0,0.0,1.0,1.0
snap_TX,1969.0,0.330117,0.470374,0.0,0.0,0.0,1.0,1.0
snap_WI,1969.0,0.330117,0.470374,0.0,0.0,0.0,1.0,1.0


date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

In [4]:
# Convert 'date' column to datetime format
calendar['date'] = pd.to_datetime(calendar['date'])
# drop weekday 
calendar = calendar.drop(columns=['weekday']) 

In [5]:
# define a function to do the following steps for columns: event_name_1, event_type_1, event_name_2, event_type_2
def process_event_column(column, calendardata):
    # Get unique values for the event_name column and the associated date
    temp_df = calendardata[[column, 'date']].dropna().sort_values(by=[column, 'date']).reset_index(drop=True)
    # Pivot the data: event names become columns, and dates remain as index
    temp_df = temp_df.pivot(index='date', columns=column, values=column)
    # Fill NaN values with 0, and existing values with 1
    temp_df = temp_df.fillna(0).applymap(lambda x: 1 if x != 0 else 0)
    # Merge the resulting pivoted DataFrame back to the original calendar data by date
    calendardata = calendardata.merge(temp_df, on='date')
    # Drop the original column after merge
    calendardata = calendardata.drop(columns=[column])
    # Return the updated calendar data and the new column names
    return calendardata, temp_df.columns.tolist()


# apply the function to the columns
calendar, event_name_1_columns = process_event_column(column='event_name_1', calendardata=calendar)
calendar, event_type_1_columns = process_event_column(column='event_type_1', calendardata=calendar)
calendar, event_name_2_columns = process_event_column(column='event_name_2', calendardata=calendar)
calendar, event_type_2_columns = process_event_column(column='event_type_2', calendardata=calendar)

print("event_name_1_columns: ", event_name_1_columns)
print("event_type_1_columns: ", event_type_1_columns)
print("event_name_2_columns: ", event_name_2_columns)
print("event_type_2_columns: ", event_type_2_columns)

event_name_1_columns:  ['Chanukah End', 'Christmas', 'Cinco De Mayo', 'ColumbusDay', 'Easter', 'Eid al-Fitr', 'EidAlAdha', "Father's day", 'Halloween', 'IndependenceDay', 'LaborDay', 'LentStart', 'LentWeek2', 'MartinLutherKingDay', 'MemorialDay', "Mother's day", 'NBAFinalsEnd', 'NBAFinalsStart', 'NewYear', 'OrthodoxChristmas', 'OrthodoxEaster', 'Pesach End', 'PresidentsDay', 'Purim End', 'Ramadan starts', 'StPatricksDay', 'SuperBowl', 'Thanksgiving', 'ValentinesDay', 'VeteransDay']
event_type_1_columns:  ['Cultural', 'National', 'Religious', 'Sporting']
event_name_2_columns:  ['Cinco De Mayo', 'Easter', "Father's day", 'OrthodoxEaster']
event_type_2_columns:  ['Cultural', 'Religious']


In [7]:
calendar.head().T
# note snap_[CA/TX/WI]: whether the stores in each state allowed purchases with SNAP food stamps at this date (1) or not (0). 

,0,1,2,3,4
date,2011-04-24 00:00:00,2013-05-05 00:00:00,2014-04-20 00:00:00,2014-06-15 00:00:00,2016-06-19 00:00:00
wm_yr_wk,11113,11315,11412,11420,11621
wday,2,2,2,2,2
month,4,5,4,6,6
year,2011,2013,2014,2014,2016
d,d_86,d_828,d_1178,d_1234,d_1969
snap_CA,0,1,0,0,0
snap_TX,0,1,0,1,0
snap_WI,0,1,0,1,0
Chanukah End,0,0,0,0,0


#### 1.2 Sell Prices Data
The store and item IDs together with the sales price of the item as a weekly average.

In [9]:
display(sell_prices.info())
display(sell_prices.head())
display(sell_prices.describe().T)
display(sell_prices.isnull().sum().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


None

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


,count,mean,std,min,25%,50%,75%,max
wm_yr_wk,6841121.0,11382.943423,148.610026,11101.00,11247.00,11411.00,11517.00,11621.00
sell_price,6841121.0,4.410952,3.408814,0.01,2.18,3.47,5.84,107.32


store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

#### 1.3 Sales_train_validation

In [10]:
display(sales_train_val.info())
display(sales_train_val.head())
display(sales_train_val.describe().T)
display(sales_train_val.isnull().sum().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


None

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


,count,mean,std,min,25%,50%,75%,max
d_1,30490.0,1.070220,5.126689,0.0,0.0,0.0,0.0,360.0
d_2,30490.0,1.041292,5.365468,0.0,0.0,0.0,0.0,436.0
d_3,30490.0,0.780026,3.667454,0.0,0.0,0.0,0.0,207.0
d_4,30490.0,0.833454,4.415141,0.0,0.0,0.0,0.0,323.0
d_5,30490.0,0.627944,3.379344,0.0,0.0,0.0,0.0,296.0
...,...,...,...,...,...,...,...,...
d_1909,30490.0,1.159167,2.876026,0.0,0.0,0.0,1.0,88.0
d_1910,30490.0,1.149000,2.950364,0.0,0.0,0.0,1.0,77.0
d_1911,30490.0,1.328862,3.358012,0.0,0.0,0.0,1.0,141.0
d_1912,30490.0,1.605838,4.089422,0.0,0.0,0.0,2.0,171.0


id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
           ..
d_1909      0
d_1910      0
d_1911      0
d_1912      0
d_1913      0
Length: 1919, dtype: int64

In [18]:
# Pivot the dataframe d1 to d1913 to rows
sales_train_val_T = sales_train_val.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    value_vars=sales_train_val.columns[sales_train_val.columns.get_loc('d_1'):sales_train_val.columns.get_loc('d_1913')+1],
    var_name='d',
    value_name='sales'
)
display(sales_train_val_T.head())
display(sales_train_val_T.tail())

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3
58327369,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1913,0
